In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from functools import partial
import sklearn.datasets
import scipy.linalg
slider_layout = widgets.Layout(width='600px', height='20px')
slider_style = {'description_width': 'initial'}
IntSlider_nice = partial(widgets.IntSlider, style=slider_style, layout=slider_layout, continuous_update=False)
FloatSlider_nice = partial(widgets.FloatSlider, style=slider_style, layout=slider_layout, continuous_update=False)
SelSlider_nice = partial(widgets.SelectionSlider, style=slider_style, layout=slider_layout, continuous_update=False)

# Python y  Rendimiento

Python es un lenguaje interpretado de alto nivel que es muy conveniente para prototipar y hacer análisis 
exploratorio

Esto tiene un costo: Menor **rendimiento** a igual **complejidad** en comparación a lenguajes compilados de bajo nivel

Podemos ser más especificos y hablar de **eficiencia**

> Temporal: Tiempo para completar una tarea (tiempo en la CPU)

> Espacial: Utilización de espacio (memoria RAM, disco)

Ambos son factores críticos en algunas aplicaciones (big-data)

Sin embargo se puede obtener un buen rendimiento en Python si usamos las librerías presentadas en este curso

> **NumPy, Scipy, Pandas** están compiladas en C y Fortran

En las siguientes clases estudiaremos como mejorar el rendimiento de un código escrito en Python

El primer paso es:


# *Profiling*

Se refiere a medir los 

1. Tiempos de ejecución (total, por función, por linea)
1. Uso de recursos (memoria, cpu, disco)

de una rutina con el fin de encontrar aquellas secciones más lentas e ineficientes (y posteriormente corregirlas)

### Ejemplo: Set de Julia

El [set de Julia](https://en.wikipedia.org/wiki/Julia_set) es un fractal asociado a la función

$$
f(z) = z^2 + c,
$$
donde $c \in \mathbb{C}$

En Python:

In [ ]:
# Ver código de make_fractal en slow_function.py
from fractal import make_fractal
fractal_image = make_fractal(N=500, maxiter=50)

fig, ax = plt.subplots(figsize=(8, 4), tight_layout=True)
ax.imshow(fractal_image, aspect='equal', cmap=plt.cm.viridis, origin='lower')
ax.axis('off');

Usaremos partial para fijar los parámetros de `make_fractal`

A continuación haremos un *profiling* de `slow_function`

In [ ]:
from functools import partial
slow_function = partial(make_fractal, N=500, maxiter=50)

## Tiempo de ejecución con magias de IPython

Podemos medir el tiempo total de un bloque de ejecución completo con la magia `%%time`

¿Cuánto demora en calcularse el set de julia?

In [ ]:
%%time 
result = slow_function()

En mi computador (Core i5-4200U, 1.6 Ghz) demora 1.35s

¿Cuánto demora en el tuyo? ¿Cómo se compara tu CPU con el mio?

También podemos medir el tiempo de una linea en particular con la magia `%time`

Por ejemplo:

In [ ]:
%time result1 = slow_function()
%time result2 = slow_function()
%time result3 = slow_function()
%time result4 = slow_function()
# Son los resultados iguales?
np.allclose(result1, result2)

A pesar de ejecutar el mismo código y obtener el mismo resultado los tiempos de cómputo son ligeramente distintos ¿Por qué?

> Cada vez que ejecutamos un código alteramos el estado de nuestro sistema (cache, memoria)

La magia `%timeit -rX -nY` ejecuta nuestro código X veces y retorna el tiempo promedio y la desviación estándar. Por cada ejecución se guarda el mejor tiempo de Y repeticiones

El tiempo para 10 repeticiones del set de julia es:

In [ ]:
%timeit -r10 -n1 result = slow_function()

Esta magia se basa en el módulo de Python [timeit](https://docs.python.org/3/library/timeit.html)

**OJO:** Los tiempos de `timeit` suelen ser menores a los de `time`. Es porque `timeit` omite las tareas de *garbage collection*

Podemos activar *gc* usando el módulo de forma directa:

    import timeit
    timeit.timeit(slow_function, 'gc.enable()', number=10)/10

### Midiendo el tiempo de cada función

El módulo de Python [profile](https://docs.python.org/3/library/profile.html) mide la cantidad de llamadas y el tiempo de cada función ejecutada por nuestra rutina

La magia de IPython `%prun` es una forma conveniente para usar este módulo

Atributos de la tabla de `prun`
- ncalls: Número de veces que se llama la función
- tottime: Tiempo total en dicha función (sin contar subfunciones)
- percall: ttime/ncalls
- cumtime: Tiempo total en dicha función y sus subfunciones (tiempo de función recursiva)
- percall: cumtime/ncalls


In [ ]:
#%prun -s cumtime slow_function(data)
%prun slow_function()

De la tabla vemos que 
- La función con mayor tiempo total es evaluate que está en la linea 10 de fractal.py
- evaluate e initialize se llaman 500.000 veces

También podemos notar que el tiempo total es mayor que el que medimos con `time` y `timeit`: Esto corresponde al overhead de `prun`

#### Visualizando con snakeviz

Alternativamente podemos visualizar los resultados de `prun` en nuestro navegador usando [`SnakeViz`](https://jiffyclub.github.io/snakeviz/)

Primero lo instalamos con [conda](https://anaconda.org/conda-forge/snakeviz) o con

    pip3 install snakeviz
    
Esto creará un ejecutable snakeviz en `/usr/bin`

Luego cargamos la extensión para jupyter:

In [ ]:
%load_ext snakeviz

Y ahora podemos usar las magias `%snakeviz` para una rutina y `%%snakeviz` para un bloque completo. La opción `-t` carga el gráfico en una pestaña de navegador nueva

In [ ]:
%snakeviz -t slow_function()

Esta herramiento puede mejorar considerablemente el estudio de nuestro código cuando se tiene una gran cantidad de funciones en distintas jerarquías

### Midiendo el tiempo de cada linea

A veces puede ser más informativo medir el tiempo linea a linea en lugar de función a función

Podemos lograr esto último usando la extensión [`line_profiler`](https://github.com/rkern/line_profiler). 

Luego de instalar y habilitarla tendremos a disposición la magia `%lprun` que funciona de forma similar a `prun`

***
Se recomiendo instalar con conda: https://anaconda.org/anaconda/line_profiler

Si no usas conda puedes instalar manualmente con

    git clone https://github.com/rkern/line_profiler.git
    find line_profiler -name '*.pyx' -exec cython {} \;
    cd line_profiler && pip3 install . --user 
    
La instalación con PIP no funciona de momento (ver repositorio de line_profiler)
***

In [ ]:
%load_ext line_profiler

La magia requiere que se especifique un método/función dentro de la rutina

In [ ]:
%lprun -f make_fractal slow_function()

Podemos ver que el 79.2% del tiempo se ocupa en la linea 26 (evaluate), mientras que un 13.8% se ocupa en la linea 25

## Uso de memoria con magias de IPython

Podemos descargar y habilitar la extensión `memory_profiler` para medir la cantidad de memoria usada por nuestra rutina

    pip3 install memory_profiler --user


In [ ]:
%load_ext memory_profiler

Podemos usar `%memit` para medir la memoria total

In [ ]:
%memit slow_function()

y `%mprun` para medir el uso de memoria linea por linea

**OJO:** `mprun` requiere que la función esté escrita en un archivo `.py` (en este caso está en fractal.py)

    %mprun -f make_fractal make_fractal(N=50, maxiter=1)